# 📚 의학/과학 논문 검색 → RAG 시스템 (확장판)

이 노트북은 **arXiv, PubMed** 등 다양한 소스에서 논문을 검색하고,
**BioBERT, PubMedBERT** 등 다양한 임베딩 모델을 선택하여 RAG 시스템을 구축합니다.

## ✨ 주요 기능

### 📖 논문 검색 소스
| 소스 | 분야 | 특징 |
|------|------|------|
| **arXiv** | CS, 물리, 수학 | 무료 PDF 제공 |
| **PubMed** | 의학, 생물학 | 3천만+ 논문 |

### 🧠 임베딩 모델
| 모델 | 특화 분야 | 특징 |
|------|----------|------|
| **all-MiniLM** | 일반 | 가볍고 빠름 |
| **BioBERT** | 의학/생물학 | 생의학 텍스트 특화 |
| **PubMedBERT** | 의학 논문 | PubMed 논문으로 학습 |
| **SciBERT** | 과학 전반 | 과학 논문 특화 |

## 전체 흐름
```
1. 검색 소스 선택 (arXiv / PubMed)
       ↓
2. 논문 검색 & PDF 다운로드
       ↓
3. 텍스트 추출 & 청킹
       ↓
4. 임베딩 모델 선택 (MiniLM / BioBERT / PubMedBERT / SciBERT)
       ↓
5. 벡터 DB 생성 (FAISS)
       ↓
6. RAG 질의응답
```

---
## 🔧 Step 0: 필요한 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install arxiv biopython pypdf2 pdfplumber -q
!pip install langchain langchain-community langchain-openai -q
!pip install faiss-cpu -q
!pip install sentence-transformers transformers -q
!pip install openai tiktoken -q
!pip install requests xmltodict -q
!pip install "numpy<2" -q  # NumPy 2.x 호환성 문제 해결

print("✅ 라이브러리 설치 완료!")

---
## 📦 Step 1: 라이브러리 임포트 및 설정

In [ ]:
# 기본 라이브러리
import os
import requests
import time
import re
from pathlib import Path
from typing import List, Dict, Optional

# 논문 검색
import arxiv
import xmltodict

# PDF 처리
from PyPDF2 import PdfReader
import pdfplumber

# LangChain 관련
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료!")

In [ ]:
# 작업 디렉토리 설정
PAPERS_DIR = "./papers"
VECTORSTORE_DIR = "./vectorstore"

# 디렉토리 생성
os.makedirs(PAPERS_DIR, exist_ok=True)
os.makedirs(VECTORSTORE_DIR, exist_ok=True)

print(f"📁 논문 저장 폴더: {PAPERS_DIR}")
print(f"📁 벡터 DB 폴더: {VECTORSTORE_DIR}")

---
## ⚙️ Step 2: 설정 선택

### 🎯 여기서 원하는 옵션을 선택하세요!

In [ ]:
#############################################
#         🎯 설정 선택 (여기를 수정하세요!)      #
#############################################

# ========== 1. 검색 소스 선택 ==========
# 'arxiv'   : 컴퓨터과학, 물리학, 수학 논문
# 'pubmed'  : 의학, 생물학 논문
# 'both'    : 둘 다 검색
SEARCH_SOURCE = 'pubmed'  # 'arxiv', 'pubmed', 'both' 중 선택

# ========== 2. 검색어 설정 ==========
SEARCH_QUERY = "COVID-19 vaccine efficacy"  # 원하는 검색어 입력
MAX_RESULTS = 10  # 검색할 논문 수

# ========== 3. 임베딩 모델 선택 ==========
# 'minilm'      : 일반 목적 (가볍고 빠름)
# 'biobert'     : 의학/생물학 특화
# 'pubmedbert'  : PubMed 논문 특화 (의학 최적)
# 'scibert'     : 과학 논문 전반
EMBEDDING_MODEL = 'pubmedbert'  # 'minilm', 'biobert', 'pubmedbert', 'scibert' 중 선택

# ========== 4. LLM 설정 (선택사항) ==========
USE_OPENAI = True# OpenAI API 사용 여부
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY_HERE"  # OpenAI API 키 (USE_OPENAI=True일 때 필요)

# ========== 5. 청킹 설정 ==========
CHUNK_SIZE = 1000  # 청크 크기 (글자 수)
CHUNK_OVERLAP = 200  # 청크 겹침

print("⚙️ 현재 설정:")
print(f"   📖 검색 소스: {SEARCH_SOURCE}")
print(f"   🔍 검색어: {SEARCH_QUERY}")
print(f"   📄 최대 논문 수: {MAX_RESULTS}")
print(f"   🧠 임베딩 모델: {EMBEDDING_MODEL}")
print(f"   🤖 OpenAI 사용: {USE_OPENAI}")

---
## 🔍 Step 3: 논문 검색 클래스 정의

In [ ]:
class PaperSearcher:
    """
    다양한 소스에서 논문을 검색하는 클래스
    지원: arXiv, PubMed
    """
    
    def __init__(self):
        self.papers = []
    
    # ==================== arXiv 검색 ====================
    def search_arxiv(self, query: str, max_results: int = 5) -> List[Dict]:
        """
        arXiv에서 논문 검색
        
        Parameters:
        -----------
        query : str - 검색어
        max_results : int - 최대 결과 수
        
        Returns:
        --------
        List[Dict] : 논문 정보 리스트
        """
        print(f"\n🔍 arXiv에서 '{query}' 검색 중...")
        
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )
        
        papers = []
        for result in search.results():
            paper = {
                'source': 'arXiv',
                'title': result.title,
                'authors': [author.name for author in result.authors],
                'abstract': result.summary,
                'pdf_url': result.pdf_url,
                'published': result.published.strftime('%Y-%m-%d'),
                'id': result.entry_id.split('/')[-1]
            }
            papers.append(paper)
            print(f"   📄 {paper['title'][:60]}...")
        
        print(f"   ✅ arXiv: {len(papers)}개 논문 발견")
        return papers
    
    # ==================== PubMed 검색 ====================
    def search_pubmed(self, query: str, max_results: int = 5) -> List[Dict]:
        """
        PubMed에서 논문 검색 (NCBI E-utilities API 사용)
        
        Parameters:
        -----------
        query : str - 검색어
        max_results : int - 최대 결과 수
        
        Returns:
        --------
        List[Dict] : 논문 정보 리스트
        """
        print(f"\n🔍 PubMed에서 '{query}' 검색 중...")
        
        # Step 1: 검색하여 PMID 목록 가져오기
        search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        search_params = {
            'db': 'pubmed',
            'term': query,
            'retmax': max_results,
            'retmode': 'json',
            'sort': 'relevance'
        }
        
        try:
            response = requests.get(search_url, params=search_params, timeout=30)
            response.raise_for_status()
            search_data = response.json()
            pmids = search_data.get('esearchresult', {}).get('idlist', [])
            
            if not pmids:
                print("   ⚠️ PubMed: 검색 결과 없음")
                return []
            
            # Step 2: PMID로 상세 정보 가져오기
            fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
            fetch_params = {
                'db': 'pubmed',
                'id': ','.join(pmids),
                'retmode': 'xml'
            }
            
            time.sleep(0.5)  # API 제한 준수
            response = requests.get(fetch_url, params=fetch_params, timeout=30)
            response.raise_for_status()
            
            # XML 파싱
            data = xmltodict.parse(response.content)
            articles = data.get('PubmedArticleSet', {}).get('PubmedArticle', [])
            
            # 단일 결과인 경우 리스트로 변환
            if isinstance(articles, dict):
                articles = [articles]
            
            papers = []
            for article in articles:
                try:
                    medline = article.get('MedlineCitation', {})
                    article_data = medline.get('Article', {})
                    
                    # 제목 추출
                    title = article_data.get('ArticleTitle', 'No Title')
                    if isinstance(title, dict):
                        title = title.get('#text', 'No Title')
                    
                    # 초록 추출
                    abstract_data = article_data.get('Abstract', {}).get('AbstractText', '')
                    if isinstance(abstract_data, list):
                        abstract = ' '.join([a.get('#text', str(a)) if isinstance(a, dict) else str(a) for a in abstract_data])
                    elif isinstance(abstract_data, dict):
                        abstract = abstract_data.get('#text', str(abstract_data))
                    else:
                        abstract = str(abstract_data) if abstract_data else 'No abstract available'
                    
                    # 저자 추출
                    author_list = article_data.get('AuthorList', {}).get('Author', [])
                    if isinstance(author_list, dict):
                        author_list = [author_list]
                    authors = []
                    for author in author_list[:5]:  # 최대 5명
                        if isinstance(author, dict):
                            last = author.get('LastName', '')
                            first = author.get('ForeName', '')
                            if last:
                                authors.append(f"{last} {first}".strip())
                    
                    # PMID
                    pmid = medline.get('PMID', {}).get('#text', 'Unknown')
                    
                    # 발행일
                    pub_date = article_data.get('Journal', {}).get('JournalIssue', {}).get('PubDate', {})
                    year = pub_date.get('Year', 'Unknown')
                    
                    paper = {
                        'source': 'PubMed',
                        'title': title,
                        'authors': authors if authors else ['Unknown'],
                        'abstract': abstract,
                        'pdf_url': None,  # PubMed는 직접 PDF 링크 제공 안 함
                        'pubmed_url': f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/",
                        'pmc_url': self._get_pmc_pdf_url(pmid),
                        'published': str(year),
                        'id': f"PMID_{pmid}"
                    }
                    papers.append(paper)
                    print(f"   📄 {paper['title'][:60]}...")
                    
                except Exception as e:
                    continue
            
            print(f"   ✅ PubMed: {len(papers)}개 논문 발견")
            return papers
            
        except Exception as e:
            print(f"   ❌ PubMed 검색 오류: {str(e)}")
            return []
    
    def _get_pmc_pdf_url(self, pmid: str) -> Optional[str]:
        """
        PubMed Central에서 무료 PDF URL 찾기
        """
        try:
            # PMC ID 확인
            link_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"
            params = {
                'dbfrom': 'pubmed',
                'db': 'pmc',
                'id': pmid,
                'retmode': 'json'
            }
            time.sleep(0.3)
            response = requests.get(link_url, params=params, timeout=10)
            data = response.json()
            
            linksets = data.get('linksets', [{}])[0]
            linksetdbs = linksets.get('linksetdbs', [])
            
            for linksetdb in linksetdbs:
                if linksetdb.get('dbto') == 'pmc':
                    pmc_ids = linksetdb.get('links', [])
                    if pmc_ids:
                        pmc_id = pmc_ids[0]
                        return f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmc_id}/pdf/"
            return None
        except:
            return None
    
    # ==================== 통합 검색 ====================
    def search(self, query: str, source: str = 'both', max_results: int = 5) -> List[Dict]:
        """
        선택한 소스에서 논문 검색
        
        Parameters:
        -----------
        query : str - 검색어
        source : str - 'arxiv', 'pubmed', 'both'
        max_results : int - 소스당 최대 결과 수
        
        Returns:
        --------
        List[Dict] : 논문 정보 리스트
        """
        papers = []
        
        if source in ['arxiv', 'both']:
            papers.extend(self.search_arxiv(query, max_results))
        
        if source in ['pubmed', 'both']:
            papers.extend(self.search_pubmed(query, max_results))
        
        self.papers = papers
        return papers

print("✅ PaperSearcher 클래스 정의 완료!")

---
## 📥 Step 4: PDF 다운로드 클래스

In [ ]:
class PDFDownloader:
    """
    논문 PDF 다운로드 클래스
    """
    
    def __init__(self, save_dir: str = PAPERS_DIR):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
    
    def download(self, paper: Dict) -> Optional[str]:
        """
        논문 PDF 다운로드
        """
        # 파일명 생성
        safe_title = re.sub(r'[^\w\s-]', '', paper['title'])[:50]
        filename = f"{paper['id']}_{safe_title}.pdf"
        filepath = os.path.join(self.save_dir, filename)
        
        # 이미 존재하면 스킵
        if os.path.exists(filepath):
            print(f"   ⏭️ 이미 존재: {filename[:40]}...")
            return filepath
        
        # PDF URL 결정
        pdf_url = paper.get('pdf_url') or paper.get('pmc_url')
        
        if not pdf_url:
            # PubMed 논문의 경우 초록만 저장
            if paper['source'] == 'PubMed':
                return self._save_abstract_as_text(paper, filename)
            print(f"   ⚠️ PDF URL 없음: {paper['title'][:40]}...")
            return None
        
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (compatible; ResearchBot/1.0)'}
            response = requests.get(pdf_url, headers=headers, timeout=60)
            response.raise_for_status()
            
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            print(f"   ✅ 다운로드: {filename[:40]}...")
            return filepath
            
        except Exception as e:
            # PDF 다운로드 실패 시 초록 저장
            print(f"   ⚠️ PDF 다운로드 실패, 초록 저장: {paper['title'][:30]}...")
            return self._save_abstract_as_text(paper, filename)
    
    def _save_abstract_as_text(self, paper: Dict, filename: str) -> Optional[str]:
        """
        PDF 없을 때 초록을 텍스트 파일로 저장
        """
        txt_filename = filename.replace('.pdf', '.txt')
        filepath = os.path.join(self.save_dir, txt_filename)
        
        content = f"""Title: {paper['title']}

Authors: {', '.join(paper['authors'])}

Source: {paper['source']}

Published: {paper['published']}

Abstract:
{paper['abstract']}

URL: {paper.get('pubmed_url', paper.get('pdf_url', 'N/A'))}
"""
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        
        return filepath
    
    def download_all(self, papers: List[Dict]) -> List[str]:
        """
        모든 논문 다운로드
        """
        print("\n📥 논문 다운로드 시작...\n")
        
        downloaded = []
        for paper in papers:
            filepath = self.download(paper)
            if filepath:
                downloaded.append(filepath)
            time.sleep(0.5)  # API 제한 준수
        
        print(f"\n📁 총 {len(downloaded)}개 파일 다운로드 완료!")
        return downloaded

print("✅ PDFDownloader 클래스 정의 완료!")

---
## 📄 Step 5: 텍스트 추출 클래스

In [ ]:
class TextExtractor:
    """
    PDF/TXT에서 텍스트 추출
    """
    
    @staticmethod
    def extract(filepath: str) -> str:
        """
        파일에서 텍스트 추출
        """
        if filepath.endswith('.txt'):
            return TextExtractor._extract_from_txt(filepath)
        elif filepath.endswith('.pdf'):
            return TextExtractor._extract_from_pdf(filepath)
        return ""
    
    @staticmethod
    def _extract_from_txt(filepath: str) -> str:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return f.read()
        except:
            return ""
    
    @staticmethod
    def _extract_from_pdf(filepath: str) -> str:
        text = ""
        try:
            # PyPDF2 시도
            reader = PdfReader(filepath)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
            
            # 텍스트가 너무 적으면 pdfplumber 시도
            if len(text) < 100:
                text = ""
                with pdfplumber.open(filepath) as pdf:
                    for page in pdf.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
        except Exception as e:
            print(f"      ⚠️ 추출 오류: {str(e)[:50]}")
        
        return text.strip()
    
    @staticmethod
    def extract_all(filepaths: List[str]) -> List[Dict]:
        """
        모든 파일에서 텍스트 추출
        """
        print("\n📄 텍스트 추출 중...\n")
        
        documents = []
        for filepath in filepaths:
            filename = os.path.basename(filepath)
            print(f"   📖 {filename[:50]}...")
            
            text = TextExtractor.extract(filepath)
            if text:
                documents.append({
                    'text': text,
                    'source': filename,
                    'filepath': filepath
                })
                print(f"      ✅ {len(text):,} 글자 추출")
            else:
                print(f"      ⚠️ 텍스트 없음")
        
        print(f"\n📊 총 {len(documents)}개 문서에서 텍스트 추출 완료!")
        return documents

print("✅ TextExtractor 클래스 정의 완료!")

---
## 🧠 Step 6: 임베딩 모델 선택 클래스

### 지원 모델
| 모델 | 설명 | 차원 |
|------|------|------|
| **all-MiniLM-L6-v2** | 일반 목적, 가볍고 빠름 | 384 |
| **BioBERT** | 생의학 텍스트 특화 | 768 |
| **PubMedBERT** | PubMed 논문 학습, 의학 최적 | 768 |
| **SciBERT** | 과학 논문 전반 | 768 |

In [ ]:
class EmbeddingModelFactory:
    """
    다양한 임베딩 모델을 선택하여 생성하는 팩토리 클래스
    """
    
    # 지원하는 모델 목록
    MODELS = {
        'minilm': {
            'name': 'sentence-transformers/all-MiniLM-L6-v2',
            'description': '일반 목적 (가볍고 빠름)',
            'dimension': 384
        },
        'biobert': {
            'name': 'dmis-lab/biobert-base-cased-v1.2',
            'description': '의학/생물학 특화',
            'dimension': 768
        },
        'pubmedbert': {
            'name': 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext',
            'description': 'PubMed 논문 특화 (의학 최적)',
            'dimension': 768
        },
        'scibert': {
            'name': 'allenai/scibert_scivocab_uncased',
            'description': '과학 논문 전반',
            'dimension': 768
        }
    }
    
    @classmethod
    def list_models(cls):
        """사용 가능한 모델 목록 출력"""
        print("\n🧠 사용 가능한 임베딩 모델:\n")
        for key, info in cls.MODELS.items():
            print(f"   • {key}: {info['description']}")
            print(f"     모델: {info['name']}")
            print(f"     차원: {info['dimension']}")
            print()
    
    @classmethod
    def create(cls, model_type: str = 'minilm', device: str = 'cpu'):
        """
        임베딩 모델 생성
        
        Parameters:
        -----------
        model_type : str
            'minilm', 'biobert', 'pubmedbert', 'scibert' 중 선택
        device : str
            'cpu' 또는 'cuda'
        
        Returns:
        --------
        HuggingFaceEmbeddings : 임베딩 모델
        """
        if model_type not in cls.MODELS:
            print(f"⚠️ 알 수 없는 모델: {model_type}. 'minilm' 사용")
            model_type = 'minilm'
        
        model_info = cls.MODELS[model_type]
        model_name = model_info['name']
        
        print(f"\n🧠 임베딩 모델 로딩 중...")
        print(f"   모델: {model_type}")
        print(f"   설명: {model_info['description']}")
        print(f"   차원: {model_info['dimension']}")
        print(f"   (처음 실행 시 모델 다운로드에 시간이 걸릴 수 있습니다)\n")
        
        try:
            embeddings = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs={'device': device},
                encode_kwargs={'normalize_embeddings': True}
            )
            print(f"✅ {model_type} 모델 로드 완료!")
            return embeddings
        
        except Exception as e:
            print(f"❌ 모델 로드 실패: {str(e)}")
            print("   기본 모델(minilm)로 대체합니다...")
            
            return HuggingFaceEmbeddings(
                model_name='sentence-transformers/all-MiniLM-L6-v2',
                model_kwargs={'device': device},
                encode_kwargs={'normalize_embeddings': True}
            )

# 사용 가능한 모델 확인
EmbeddingModelFactory.list_models()

---
## 🔧 Step 7: RAG 시스템 클래스

In [ ]:
class RAGSystem:
    """
    RAG (Retrieval-Augmented Generation) 시스템
    """
    
    def __init__(self, embeddings, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.embeddings = embeddings
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectorstore = None
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
    
    def build_vectorstore(self, documents: List[Dict]) -> FAISS:
        """
        문서들로 벡터 스토어 생성
        """
        print("\n✂️ 텍스트 청킹 중...")
        
        all_chunks = []
        all_metadata = []
        
        for doc in documents:
            chunks = self.text_splitter.split_text(doc['text'])
            for i, chunk in enumerate(chunks):
                all_chunks.append(chunk)
                all_metadata.append({
                    'source': doc['source'],
                    'chunk_id': i
                })
            print(f"   📄 {doc['source'][:40]}...: {len(chunks)} 청크")
        
        print(f"\n📊 총 청크 수: {len(all_chunks)}개")
        
        print("\n💾 벡터 DB 생성 중... (시간이 걸릴 수 있습니다)")
        
        self.vectorstore = FAISS.from_texts(
            texts=all_chunks,
            embedding=self.embeddings,
            metadatas=all_metadata
        )
        
        print("✅ 벡터 DB 생성 완료!")
        return self.vectorstore
    
    def save_vectorstore(self, path: str = VECTORSTORE_DIR):
        """벡터 스토어 저장"""
        if self.vectorstore:
            self.vectorstore.save_local(path)
            print(f"💾 벡터 DB 저장 완료: {path}")
    
    def load_vectorstore(self, path: str = VECTORSTORE_DIR):
        """벡터 스토어 로드"""
        self.vectorstore = FAISS.load_local(
            path,
            self.embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"✅ 벡터 DB 로드 완료: {path}")
        return self.vectorstore
    
    def search(self, query: str, k: int = 3) -> List[Dict]:
        """
        유사 문서 검색
        """
        if not self.vectorstore:
            print("❌ 벡터 스토어가 없습니다. 먼저 build_vectorstore()를 실행하세요.")
            return []
        
        docs_with_scores = self.vectorstore.similarity_search_with_score(query, k=k)
        
        results = []
        for doc, score in docs_with_scores:
            results.append({
                'content': doc.page_content,
                'source': doc.metadata.get('source', 'Unknown'),
                'score': float(score)
            })
        
        return results
    
    def answer(self, question: str, k: int = 3) -> Dict:
        """
        질문에 대한 답변 생성 (검색 결과 기반)
        """
        results = self.search(question, k=k)
        
        return {
            'question': question,
            'contexts': results,
            'sources': list(set([r['source'] for r in results]))
        }

print("✅ RAGSystem 클래스 정의 완료!")

---
## 🚀 Step 8: 전체 파이프라인 실행!

In [ ]:
# ==================== 1. 논문 검색 ====================
print("=" * 60)
print("📚 Step 1: 논문 검색")
print("=" * 60)

searcher = PaperSearcher()
papers = searcher.search(
    query=SEARCH_QUERY,
    source=SEARCH_SOURCE,
    max_results=MAX_RESULTS
)

print(f"\n📊 검색 결과: 총 {len(papers)}개 논문")

In [ ]:
# 검색 결과 상세 보기
print("\n📋 검색된 논문 목록:\n")
for i, paper in enumerate(papers, 1):
    print(f"[{i}] {paper['source']} | {paper['title'][:70]}...")
    print(f"    저자: {', '.join(paper['authors'][:3])}")
    print(f"    발행: {paper['published']}")
    print()

In [ ]:
# ==================== 2. PDF 다운로드 ====================
print("=" * 60)
print("📥 Step 2: PDF 다운로드")
print("=" * 60)

downloader = PDFDownloader(PAPERS_DIR)
downloaded_files = downloader.download_all(papers)

In [ ]:
# ==================== 3. 텍스트 추출 ====================
print("=" * 60)
print("📄 Step 3: 텍스트 추출")
print("=" * 60)

documents = TextExtractor.extract_all(downloaded_files)

In [ ]:
# ==================== 4. 임베딩 모델 로드 ====================
print("=" * 60)
print("🧠 Step 4: 임베딩 모델 로드")
print("=" * 60)

embeddings = EmbeddingModelFactory.create(
    model_type=EMBEDDING_MODEL,
    device='cpu'  # GPU 있으면 'cuda'로 변경
)

In [ ]:
# ==================== 5. RAG 시스템 구축 ====================
print("=" * 60)
print("💾 Step 5: RAG 시스템 구축")
print("=" * 60)

rag = RAGSystem(
    embeddings=embeddings,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# 벡터 스토어 생성
vectorstore = rag.build_vectorstore(documents)

# 저장
rag.save_vectorstore(VECTORSTORE_DIR)

---
## 💬 Step 9: 질의응답 테스트

In [ ]:
def ask(question: str, k: int = 3):
    """
    질문하고 답변 받기
    """
    print(f"\n{'='*60}")
    print(f"❓ 질문: {question}")
    print(f"{'='*60}")
    
    result = rag.answer(question, k=k)
    
    print(f"\n📚 관련 문서 {len(result['contexts'])}개 검색됨:\n")
    
    for i, ctx in enumerate(result['contexts'], 1):
        similarity = 1 / (1 + ctx['score'])  # 거리를 유사도로 변환
        print(f"[{i}] 출처: {ctx['source'][:50]}")
        print(f"    유사도: {similarity:.2%}")
        print(f"    내용: {ctx['content'][:300]}...")
        print("-" * 40)
    
    print(f"\n📖 참고 문서: {', '.join(result['sources'])}")
    print(f"{'='*60}\n")
    
    return result

In [ ]:
# 🎯 질문 테스트!
ask("What is the efficacy of COVID-19 vaccines?")

In [ ]:
# 다른 질문 테스트
ask("What are the side effects?")

In [ ]:
# 🎯 직접 질문해보세요!
YOUR_QUESTION = "What is the mechanism of action?"  # 원하는 질문으로 변경
ask(YOUR_QUESTION)

---
## 🤖 Step 10: OpenAI LLM 연동 (선택사항)

OpenAI API를 사용하면 검색된 문서를 바탕으로 자연스러운 답변을 생성할 수 있습니다.

In [ ]:
if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    
    # LLM 설정
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0
    )
    
    # 프롬프트 템플릿
    prompt_template = """
    당신은 의학/과학 논문을 분석하는 전문가입니다.
    다음 문맥(Context)을 사용하여 질문에 답변하세요.
    답을 모르면 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 말하세요.
    
    Context: {context}
    
    Question: {question}
    
    Answer (한국어로):
    """
    
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    
    # RAG 체인 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    print("✅ OpenAI RAG 체인 생성 완료!")
    
    def ask_llm(question: str):
        """OpenAI LLM을 사용한 질의응답"""
        print(f"\n{'='*60}")
        print(f"❓ 질문: {question}")
        print(f"{'='*60}")
        
        result = qa_chain({"query": question})
        
        print(f"\n🤖 AI 답변:\n{result['result']}")
        print(f"\n📖 참고 문서:")
        for i, doc in enumerate(result['source_documents'], 1):
            print(f"   [{i}] {doc.metadata.get('source', 'Unknown')[:50]}")
        
        return result
else:
    print("ℹ️ OpenAI를 사용하지 않습니다.")
    print("   USE_OPENAI = True로 설정하고 API 키를 입력하면 사용할 수 있습니다.")

In [ ]:
# OpenAI LLM 질문 (USE_OPENAI=True인 경우만 작동)
if USE_OPENAI:
    ask_llm("COVID-19 백신의 효능은 어떻게 되나요?")

---
## 🔄 Step 11: 저장된 벡터 DB 재사용

In [ ]:
# 나중에 저장된 벡터 DB 불러오기
def load_existing_rag(embedding_model: str = 'pubmedbert'):
    """
    저장된 벡터 DB를 불러와서 RAG 시스템 재구성
    """
    # 임베딩 모델 로드 (저장할 때 사용한 것과 동일해야 함!)
    embeddings = EmbeddingModelFactory.create(embedding_model)
    
    # RAG 시스템 생성
    rag = RAGSystem(embeddings)
    
    # 벡터 스토어 로드
    rag.load_vectorstore(VECTORSTORE_DIR)
    
    return rag

# 예시: 저장된 DB 불러오기
# rag = load_existing_rag('pubmedbert')
# ask("Your question here")

---
## 📊 요약

### 🎯 완성된 기능

| 기능 | 옵션 |
|------|------|
| **검색 소스** | arXiv, PubMed, 둘 다 |
| **임베딩 모델** | MiniLM, BioBERT, PubMedBERT, SciBERT |
| **LLM** | OpenAI (선택) |

### 🧠 임베딩 모델 선택 가이드

| 분야 | 추천 모델 |
|------|----------|
| 일반/빠른 테스트 | minilm |
| 의학/생물학 | biobert, pubmedbert |
| 과학 전반 | scibert |
| PubMed 논문 | pubmedbert (최적) |

### 📖 검색 소스 선택 가이드

| 분야 | 추천 소스 |
|------|----------|
| 컴퓨터 과학 | arxiv |
| 물리학/수학 | arxiv |
| 의학/생물학 | pubmed |
| 종합 검색 | both |

---
## 🎉 완료!

이제 다양한 소스와 모델을 선택하여 논문 RAG 시스템을 구축할 수 있습니다!

### 💡 활용 팁

1. **의학 논문 분석**: `SEARCH_SOURCE='pubmed'`, `EMBEDDING_MODEL='pubmedbert'`
2. **AI 논문 분석**: `SEARCH_SOURCE='arxiv'`, `EMBEDDING_MODEL='scibert'`
3. **빠른 테스트**: `EMBEDDING_MODEL='minilm'`

### 🔗 참고 링크
- [arXiv](https://arxiv.org/)
- [PubMed](https://pubmed.ncbi.nlm.nih.gov/)
- [BioBERT](https://github.com/dmis-lab/biobert)
- [PubMedBERT](https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext)